Imports

In [1]:
import os
import cv2
import random
from tqdm import tqdm
import glob
from pathlib import Path
import shutil
from os import listdir
from os.path import join, isfile
from deepface import DeepFace
import numpy as np
from PIL import Image 

In [3]:
import os
APP_FOLDER = "C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB"
totalFiles = 0
totalDir = 0
for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        totalFiles += 1

print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles)) 

Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Eckhart
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Eckhart\0
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Eckhart\1
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Eckhart\2
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Guiel
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Guiel\5
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Sorkin
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Sorkin\0
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Sorkin\3
Searching in :  C:/Users/Tychon Bos/Documents/YouTubeFaces/aligned_images_DB\Aaron_Tippin
Searching

In [3]:
influencers = ['Airrack', 'BellaPoarch', 'Larry', 'LexiHensler', 'LoganPaul', 'MarkRober', 'MrBeast', 'PiersonWodzynski', 'Preston', 'Quackity', 'RosannaPansino', 'RyanTrahan', 'SoffieDossi', 'ZachKing', 'Zhc' ]


Renaming folders

In [ ]:
def rename(folder):

    for count, filename in enumerate(os.listdir(folder)):
        dst = f"Image{str(count)}.jpg"
        src =f"{folder}/{filename}"  
        dst =f"{folder}/{dst}"
        
        os.rename(src, dst)
        

In [ ]:
rename("C:/Users/Tychon Bos/Documents/test_siamese")

List all subfolders in YouTubeFaces/frame_images_DB

In [43]:
subfolders = [ f.path for f in os.scandir("C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/") if f.is_dir() ]
print(subfolders[:5])
all_names = []
for name in subfolders:
    name = name.split("/")
    name = name[-1]
    all_names.append(name)
    
print(len(all_names))


['C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Aaron_Eckhart', 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Aaron_Guiel', 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Aaron_Sorkin', 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Aaron_Tippin', 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Abbas_Kiarostami']
1595


In [ ]:
subfolders_youtube_training = [ f.path for f in os.scandir("C:/Users/Tychon Bos/Documents/ifd_training/") if f.is_dir() ]
subfolders_youtube_testing = [ f.path for f in os.scandir("C:/Users/Tychon Bos/Documents/ifd_testing/") if f.is_dir() ]
print(subfolders_youtube_training)
print(subfolders_youtube_testing)

List all subfolders in YouTubeFaces/frame_images_DB with number of sub directories

In [5]:
dcount = []

for i in range(len(subfolders)):
    num_of_sub_dirs = 0
    current_folder = subfolders[i]
    path = Path(current_folder)
    for f in path.glob('*'): 
        if f.is_dir():
            num_of_sub_dirs += 1
        
    dcount.append(str(path) + ' ' + str(num_of_sub_dirs))


In [12]:
dcount[:5]
print(len(dcount))

1595


Filter all directories that have 2 or more subdirectories

In [4]:
def filter2OrMore(dcountList):
    filtered = []
    with open('C:/Users/Tychon Bos/Documents/2OrMore.txt', 'w') as f:
        for path in dcountList:
            if '2' in path or '3' in path or '4' in path or '5' in path or '6' in path or '7' in path or '8' in path or '9' in path:
                org_string = path
                size = len(org_string)
                mod_string = org_string[:size - 2]
                #mod_string.replace("\\", "/")
                # replace strings in text file manually
                f.write(mod_string)
                f.write('\n')
                filtered.append(path)
    return filtered

In [ ]:
#COPY MANUALLY ALL PATHS FROM 2OrMore.txt TO allpaths.txt

In [49]:
allpaths = []
with open('C:/Users/Tychon Bos/Documents/allpaths.txt', 'r') as f:
    for line in f:
        line = line.strip()
        allpaths.append(line)

all_names_2ormore = []
for name in allpaths:
    name = name.split('/')
    name = name[-1]
    all_names_2ormore.append(name)
print(all_names_2ormore[:5])
print(len(all_names_2ormore))



['Aaron_Eckhart', 'Aaron_Sorkin', 'Aaron_Tippin', 'Abba_Eban', 'Abdel_Aziz_Al-Hakim']
1004


All persons that appear in 1 video

In [57]:
all_names #1595 names
all_names_2ormore #1004 names

all_names_1_dir = []
path = 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/'
for name in all_names:
    if name not in all_names_2ormore:
        name = path + name
        all_names_1_dir.append(name)

print(all_names_1_dir[:5])
print(len(all_names_1_dir))

all_names_1_dirdst = []
pathdst = 'C:/Users/Tychon Bos/Documents/Youtube1person/'

for name in all_names_1_dir:
    name = name.split('/')
    name = name[-1]
    name = pathdst + name
    all_names_1_dirdst.append(name)

print(all_names_1_dirdst[:5])

['C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Aaron_Guiel', 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Abbas_Kiarostami', 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Abdullah', 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Abdullah_Ahmad_Badawi', 'C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Abdul_Majeed_Shobokshi']
591
['C:/Users/Tychon Bos/Documents/Youtube1person/Aaron_Guiel', 'C:/Users/Tychon Bos/Documents/Youtube1person/Abbas_Kiarostami', 'C:/Users/Tychon Bos/Documents/Youtube1person/Abdullah', 'C:/Users/Tychon Bos/Documents/Youtube1person/Abdullah_Ahmad_Badawi', 'C:/Users/Tychon Bos/Documents/Youtube1person/Abdul_Majeed_Shobokshi']


Create lists with all paths

In [36]:
source2OrMore = []
dirnames30 = []
dirnames30_grey = []
cropped30 = []
cropped30_dlib = []
test10 = []
test10_dlib = []
test2 = []
test2_dlib = []
test2_haar, test10_haar, test30_haar = [], [], []

for name in allpaths:
    z = name.split('/')
    source2OrMore.append('C:/Users/Tychon Bos/Documents/' + 'YouTube_2OrMore' + '/' + z[-1])
    dirnames30.append('C:/Users/Tychon Bos/Documents/' + 'YouTube_2OrMore30IMG' + '/' + z[-1])
    
    dirnames30_grey.append('C:/Users/Tychon Bos/Documents/' + 'YouTube_2OrMore30IMG_grey' + '/' + z[-1])
    
    cropped30.append('C:/Users/Tychon Bos/Documents/' + 'YouTube_2OrMore30IMG_Cropped' + '/' + z[-1]) 
    
    cropped30_dlib.append('C:/Users/Tychon Bos/Documents/' + 'YouTube_2OrMore30IMG_Cropped_Dlib' + '/' + z[-1]) 
    
    test2.append('C:/Users/Tychon Bos/Documents/' + 'test2' + '/' + z[-1])
    test2_dlib.append('C:/Users/Tychon Bos/Documents/' + 'test2_cropped_dlib' + '/' + z[-1]) 
    
    test2_haar.append('C:/Users/Tychon Bos/Documents/' + 'test2_cropped_haar' + '/' + z[-1]) 
    test10_haar.append('C:/Users/Tychon Bos/Documents/' + 'test10_cropped_haar' + '/' + z[-1]) 
    test30_haar.append('C:/Users/Tychon Bos/Documents/' + 'test30_cropped_haar' + '/' + z[-1]) 
    
    test10.append('C:/Users/Tychon Bos/Documents/' + 'test10' + '/' + z[-1])
    test10_dlib.append('C:/Users/Tychon Bos/Documents/' + 'test10_cropped_dlib' + '/' + z[-1]) 


['Aaron_Eckhart', 'Aaron_Sorkin', 'Aaron_Tippin', 'Abba_Eban', 'Abdel_Aziz_Al-Hakim']
1004


Create directories

In [58]:
# Create directory

def createDirectory(dirnamesList):
    for i in range(len(dirnamesList)):
        dirName = dirnamesList[i]
        try:
        # Create target Directory
            os.mkdir(dirName)
            print("Directory " , dirName ,  " Created ") 
        except FileExistsError:
            print("Directory " , dirName ,  " already exists")

In [59]:
createDirectory(all_names_1_dirdst)

Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Aaron_Guiel  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Abbas_Kiarostami  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Abdullah  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Abdullah_Ahmad_Badawi  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Abdul_Majeed_Shobokshi  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Abdul_Rahman  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Abid_Hamid_Mahmud_Al-Tikriti  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Abraham_Foxman  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Adam_Scott  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Adel_Al-Jubeir  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Adisai_Bodharamik  Created 
Directory  C:/Users/Tychon Bos/Documents/Youtube1person/Adoor_Gopalakarishnan  Created 
Di

Copy from source to destination

In [60]:
#copy from source to destination
# src all_names_1_dir, dst all_names_1_dirdst

""" for i, j in zip(range(len(all_names_1_dir)), range(len(all_names_1_dirdst))):
    current_src = allpaths[i] 
    print('Current src: ', current_src)
    current_dst = all_names_1_dirdst[j]
    print('Current dst: ', current_dst)
    for root, dirs, files in os.walk(current_src):  #starting directory
        for file in files:
            path_file = os.path.join(root, file)
            shutil.copy2(path_file, current_dst) #destination dir """


Current src:  C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Aaron_Eckhart
Current dst:  C:/Users/Tychon Bos/Documents/Youtube1person/Aaron_Guiel
Current src:  C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Aaron_Sorkin
Current dst:  C:/Users/Tychon Bos/Documents/Youtube1person/Abbas_Kiarostami
Current src:  C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Aaron_Tippin
Current dst:  C:/Users/Tychon Bos/Documents/Youtube1person/Abdullah
Current src:  C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Abba_Eban
Current dst:  C:/Users/Tychon Bos/Documents/Youtube1person/Abdullah_Ahmad_Badawi
Current src:  C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Abdel_Aziz_Al-Hakim
Current dst:  C:/Users/Tychon Bos/Documents/Youtube1person/Abdul_Majeed_Shobokshi
Current src:  C:/Users/Tychon Bos/Documents/YouTubeFaces/frame_images_DB/Abdoulaye_Wade
Current dst:  C:/Users/Tychon Bos/Documents/Youtube1person/Abdul_Rahman
Current src:  C:/Users/Tych

Select random images

In [1]:
def randomImagePicker(sourceList, destinationList, numberOfImages):

    for src, dst in tqdm(zip(range(len(sourceList)), range(len(destinationList)))):
        src_dir = sourceList[src]
        dst_dir = destinationList[dst]
        #print('src: ', src_dir)
        #print('dst: ', dst_dir)
        file_list = os.listdir(src_dir)
        #print('file list: ', file_list)
        #print(len(file_list))
        random_list = random.sample(file_list, numberOfImages)
        #print('random list: ', random_list)

        for i in range(len(random_list)):
            current_file = random_list[i]
            #print('src: ',src_dir + "/" + current_file)
            #print('dst ',dst_dir  + "/" + current_file)
            shutil.copy(src_dir + "/" + current_file, dst_dir + "/" + current_file)


In [16]:
#randomImagePicker(dirnames30, test2, 2)

1004it [00:03, 282.31it/s]


Resize images

In [ ]:
def resize(dirlist, width, height):
    for i in range(len(dirlist)):
        current_dir = dirlist[i]
        #print('current_dir', current_dir)

        images = Path(current_dir).glob("*.jpg")
        image_strings = [str(p) for p in images]

        for image in image_strings:
            image = image.replace('\\','/')
            #print('current img before reading', image)
            img = cv2.imread(image, cv2.IMREAD_UNCHANGED) # read image
            #print('image in imread function', img)
            resize_width = width
            resize_height = height
            dim = (resize_height, resize_width)

            resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

            cv2.imwrite(image, resized)

In [ ]:
#resize(subfolders_youtube_testing, 256, 256)

Convert to grayscale

In [ ]:
def to_gray(pathlist):
    for path in pathlist:
        files = [f for f in listdir(path) if isfile(join(path,f))] 
        #print(path, files)

        for image in files:
            img = os.path.join(path, image)
            img = img.replace('\\','/')
            #print(img)
            try:
                current_img_gray = cv2.imread(img)
                gray = cv2.cvtColor(current_img_gray, cv2.COLOR_BGR2GRAY)
                cv2.imwrite(img, gray)
            except:
                print ("{} is not converted".format(img))


In [2]:
subfolders_if = [ f.path for f in os.scandir("C:/Users/Tychon Bos/Documents/influencer_data") if f.is_dir() ]
print(subfolders_if)

['C:/Users/Tychon Bos/Documents/influencer_data\\Airrack', 'C:/Users/Tychon Bos/Documents/influencer_data\\BellaPoarch', 'C:/Users/Tychon Bos/Documents/influencer_data\\Larry', 'C:/Users/Tychon Bos/Documents/influencer_data\\LexiHensler', 'C:/Users/Tychon Bos/Documents/influencer_data\\LoganPaul', 'C:/Users/Tychon Bos/Documents/influencer_data\\MarkRober', 'C:/Users/Tychon Bos/Documents/influencer_data\\MrBeast', 'C:/Users/Tychon Bos/Documents/influencer_data\\PiersonWodzynski', 'C:/Users/Tychon Bos/Documents/influencer_data\\Preston', 'C:/Users/Tychon Bos/Documents/influencer_data\\Quackity', 'C:/Users/Tychon Bos/Documents/influencer_data\\RosannaPansino', 'C:/Users/Tychon Bos/Documents/influencer_data\\RyanTrahan', 'C:/Users/Tychon Bos/Documents/influencer_data\\SoffieDossi', 'C:/Users/Tychon Bos/Documents/influencer_data\\ZachKing', 'C:/Users/Tychon Bos/Documents/influencer_data\\Zhc']


In [ ]:
namelist = []
for name in cropped30:
    name = name.split('/')
    namelist.append(name[-1])

Haar facedetection


In [42]:
#take from dirnames30, dirnames50 and dirnames70
#save in cropped30, cropped50, cropped70

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def haar_facedetection(srcpath, dstpath):
    #take paths from srcpath, detect face and save face to dstpath
    count = 0
    for src, dst in zip(srcpath, dstpath):
        currentsrc = src
        currentdst = dst
        for filepath in glob.iglob(currentsrc + '/*.jpg'):
            filepath = filepath.replace('\\','/')
            filename = filepath.split('/')[-1]

            img = cv2.imread(filepath)
            faces=face_cascade.detectMultiScale(img, 1.1, 4)

            savepath = currentdst + '/' + filename
            if faces == ():
                no_face = Image.open(filepath)
                no_face.save(savepath)
                count += 1
                
            for x, y, w, h in faces:
                face = img[y:y+h, x:x+w]
                cv2.imwrite(savepath, face)
                x=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
    return 'Could not detect: ' + str(count) + ' faces'

In [43]:
haar_facedetection(dirnames30, test30_haar)

'Could not detect: 3969 faces'

Dlib facedetection


In [20]:
def dlib_facedetection(srcpath, dstpath):
    for src, dst in zip(srcpath, dstpath):
            currentsrc = src
            currentdst = dst
            for filepath in glob.iglob(currentsrc + '/*.jpg'):
                filepath = filepath.replace('\\','/')
                filename = filepath.split('/')[-1]
                img = DeepFace.detectFace(filepath, detector_backend= 'dlib', enforce_detection=False)
                detected_face = img * 255
                savepath = currentdst + '/' + filename
                cv2.imwrite(savepath, detected_face[:, :, ::-1])

In [21]:
#dlib_facedetection(test10, test10_dlib)

In [41]:
# dlib_facedetection(dirnames50, cropped50_dlib)
# dlib_facedetection(dirnames70, cropped70_dlib)

In [ ]:
"""
for i in range(len(cropp)):
    txt_folder_path = cropp[i] 
    with open(txt_folder_path) as f_text:
        for line in f_text.readlines():
            inhoud = line.split(',')
            filename, x, y, width, height = inhoud[0], inhoud[2], inhoud[3], inhoud[4], inhoud[5]
            with open('C:/Users/Tychon Bos/Documents/processed_txt/test.txt', 'a') as f: 
                f.write(filename + ' ')
                f.write(x + ' ')
                f.write(y + ' ')
                f.write(width + ' ')
                f.write(height + ' ')
                f.write('\n')
"""